In [ ]:
# !pip install vaderSentiment

In [ ]:

import pandas as pd
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from bs4 import BeautifulSoup

df=pd.read_csv('twitter_senti.csv')


In [ ]:
corp = df[['Content','Article_id']]

## Preprocessing Text - Lowercase, Remove Numbers, Remove punctuations, White Space, Stop Words and Spars terms

In [ ]:

from nltk.corpus import stopwords
stops = set(stopwords.words('english'))
def content_to_words(content):
    review_text = BeautifulSoup(review,'lxml').get_text().lower() #Case lowering
    
    # Replacing Similies and frown faces,! and ? with coded word SM{int} in case they are valuable
    review_text = re.sub("(:\))",'r','SM1',review_text)
    review_text = re.sub(":(:\())",'r','SM2',review_text)
    review_text = re.sub("(!)",'r','SM3',review_text)
    review_text = re.sub("(\?)",'r','SM4',review_text)
    
    #Keep 'not' and next word as negation may be important
    review_text = re.sub(r'not\s\b(.*?)\b',r'not_\1',review_text)
    
    #keep letters and coded words, replace rest with whitespace
    nonnumbers_only = re.sub('[^a-zA-Z\_(SM\d)]',' ',review_text)
    
    #split into individual words on whitespace
    words = nonnumbers_only.split()
    
    #Remove stop words
    words = [w for w in words if not w in stops]
    
    return words


In [ ]:
fin_ = pd.DataFrame({'index':[], 'Content':[], 'senti_score':[],'final_sentiment':[]})

sid = SentimentIntensityAnalyzer()

for sent in corp.index:
    try:
        str_ = ' '.join(map(str,content_to_words(sent)))
        pol = sid.polarity_scores(str(str_))
        new_pol = {k: v for k, v in pol.items() if k !='compound'} # to filter only neg, neu and pos
        max_po = max(new_pol, key=new_pol.get)
        fin_ = fin_.append({'index':sent, 'Content':corp['Content'][sent], 'senti_score':pol,'final_sentiment':max_po}, ignore_index=True)
        
    except:
        pass
    
fin_.to_csv('sentiment scored.csv')